# Sympy code to compute the inner expansion

### Zoïs Moitier (2021)
            
_Karlsruhe Institute of Technology, Germany_

In [1]:
import sympy as sy
from IPython.display import display

In [2]:
order = 3  # order of the expansion

In [3]:
ε, δ, h = sy.symbols("varepsilon delta h", real=True, positive=True)
s, t, X = sy.symbols("s t X", real=True)
μ_term = sy.symbols(f"mu0:{order}", cls=sy.Function)

## The Laplace Kernel $K^L(s, t; \varepsilon)$

In [4]:
εKL = -(ε ** 2) / (2 * sy.pi * (1 + ε ** 2 + (1 - ε ** 2) * sy.cos(s + t)))
display(εKL)

-varepsilon**2/(2*pi*(varepsilon**2 + (1 - varepsilon**2)*cos(s + t) + 1))

## Expansion of the kernel $K^L(s, \pi - s + \varepsilon X; \varepsilon)$

In [5]:
εKL_series = εKL.subs(t, sy.pi - s + ε * X).series(x=ε, x0=0, n=order).expand()
display(εKL_series)

-1/(pi*X**2 + 4*pi) - X**2*varepsilon**2/(pi*X**4 + 8*pi*X**2 + 16*pi) - X**4*varepsilon**2/(12*pi*X**4 + 96*pi*X**2 + 192*pi) + O(varepsilon**3)

## Expansion of the function $\mu(\pi - s + \varepsilon X)$

In [6]:
μ_series = sum((ε * X) ** i * μ_(sy.pi - s) for i, μ_ in enumerate(μ_term)) + sy.O(
    ε ** order
)
display(μ_series)

mu0(pi - s) + X*varepsilon*mu1(pi - s) + X**2*varepsilon**2*mu2(pi - s) + O(varepsilon**3)

## Expansion of the integrand $K^L(s, \pi-s+\varepsilon X; \varepsilon) \, \mu(\pi - s + \varepsilon X)$

In [7]:
expr = (εKL_series * μ_series).expand()
display(expr)

-mu0(pi - s)/(pi*X**2 + 4*pi) - X*varepsilon*mu1(pi - s)/(pi*X**2 + 4*pi) - X**2*varepsilon**2*mu2(pi - s)/(pi*X**2 + 4*pi) - X**2*varepsilon**2*mu0(pi - s)/(pi*X**4 + 8*pi*X**2 + 16*pi) - X**4*varepsilon**2*mu0(pi - s)/(12*pi*X**4 + 96*pi*X**2 + 192*pi) + O(varepsilon**3)

## Inner expansion $\int_{-\frac{\delta}{2\varepsilon}}^{\frac{\delta}{2\varepsilon}} K^L(s, \pi-s+\varepsilon X; \varepsilon) \, \mu(\pi - s + \varepsilon X) \, \mathrm{d}X$

In [8]:
int_expr = sy.O(ε ** 2)
for i in range(order):
    expr_term = expr.coeff(ε, i)
    for u_ in μ_term:
        expr_term_u = expr_term.coeff(u_(sy.pi - s), 1).factor()
        int_term = sy.integrate(expr_term_u, (X, -δ / (2 * ε), δ / (2 * ε))).expand()
        int_expr += (int_term * u_(sy.pi - s) * ε ** i).factor().expand()

display(int_expr)

-mu0(pi - s)*atan(delta/(4*varepsilon))/pi - delta*varepsilon*mu2(pi - s)/pi - delta**3*varepsilon*mu0(pi - s)/(12*pi*delta**2 + 192*pi*varepsilon**2) + O(varepsilon**2)